In [1]:
import logging
import os
import requests
import pandas as pd
import duckdb

logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [2]:
STORE_ENDPOINT = "https://fakestoreapi.com/products/category/women%27s%20clothing"

r = requests.get(STORE_ENDPOINT).json()
store_items = pd.DataFrame(r)

#Explode "rating" (dict to columns)
store_items = store_items.join(pd.DataFrame(store_items["rating"].tolist())[["rate", "count"]])
store_items.drop("rating", axis=1, inplace=True)

store_items.rename(columns={"price": "price_usd", "rate": "rating_rate", "count": "rating_count"}, inplace=True)

store_items.head()

,id,title,price_usd,description,category,image,rating_rate,rating_count
0,15,BIYLACLESEN Women's 3-in-1 Snowboard Jacket Wi...,56.99,"Note:The Jackets is US standard size, Please c...",women's clothing,https://fakestoreapi.com/img/51Y5NI-I5jL._AC_U...,2.6,235
1,16,Lock and Love Women's Removable Hooded Faux Le...,29.95,100% POLYURETHANE(shell) 100% POLYESTER(lining...,women's clothing,https://fakestoreapi.com/img/81XH0e8fefL._AC_U...,2.9,340
2,17,Rain Jacket Women Windbreaker Striped Climbing...,39.99,Lightweight perfet for trip or casual wear---L...,women's clothing,https://fakestoreapi.com/img/71HblAHs5xL._AC_U...,3.8,679
3,18,MBJ Women's Solid Short Sleeve Boat Neck V,9.85,"95% RAYON 5% SPANDEX, Made in USA or Imported,...",women's clothing,https://fakestoreapi.com/img/71z3kpMAYsL._AC_U...,4.7,130
4,19,Opna Women's Short Sleeve Moisture,7.95,"100% Polyester, Machine wash, 100% cationic po...",women's clothing,https://fakestoreapi.com/img/51eg55uWmdL._AC_U...,4.5,146


In [3]:
def get_file_paths(directory):
    # get full path and basename for all files in directory
    files = []
    for entry in os.scandir(f'{directory}/'):
        if entry.is_file():
            files.append({"path": entry.path, "name": os.path.basename(entry.path)})
        else:
            # if the entry is a directory, recursively call get_file_paths
            files.extend(get_file_paths(entry.path))
    return files

In [4]:
# Get ecommerce data
dates = ['2024-02-12', '2024-02-13']

data_path = '../data'
rates_path = f'{data_path}/exchange_rates'

rate_files = get_file_paths(rates_path)

# I'm pulling all the rates datasets, but in a real scenario I would only pull the required dates and currencies (for example, using AWS S3 and 
# the AWS Athena query engine over parquet files)
rates_df = pd.concat((pd.read_csv(f["path"]) for f in rate_files), ignore_index=True)

# Only keep USD rates for the required dates
rates_df = rates_df[(rates_df["date"].isin(dates)) & (rates_df["from_currency"] == "USD")]
# Drop unneeded columns
rates_df.drop(columns=['from_currency', 'amount', 'to_currency','updated_at'], inplace=True)
rates_df.rename(columns={'date': 'exchange_rate_date'}, inplace=True)

rates_df.head()

,exchange_rate_date,exchange_rate
28,2024-02-12,0.92825
58,2024-02-13,0.92653


In [5]:
products_df = store_items.merge(rates_df, how='cross')
products_df["price_eur"] = products_df["price_usd"] * products_df["exchange_rate"]
print(products_df.columns)
products_df.head()

Index(['id', 'title', 'price_usd', 'description', 'category', 'image',
       'rating_rate', 'rating_count', 'exchange_rate_date', 'exchange_rate',
       'price_eur'],
      dtype='object')


,id,title,price_usd,description,category,image,rating_rate,rating_count,exchange_rate_date,exchange_rate,price_eur
0,15,BIYLACLESEN Women's 3-in-1 Snowboard Jacket Wi...,56.99,"Note:The Jackets is US standard size, Please c...",women's clothing,https://fakestoreapi.com/img/51Y5NI-I5jL._AC_U...,2.6,235,2024-02-12,0.92825,52.900968
1,15,BIYLACLESEN Women's 3-in-1 Snowboard Jacket Wi...,56.99,"Note:The Jackets is US standard size, Please c...",women's clothing,https://fakestoreapi.com/img/51Y5NI-I5jL._AC_U...,2.6,235,2024-02-13,0.92653,52.802945
2,16,Lock and Love Women's Removable Hooded Faux Le...,29.95,100% POLYURETHANE(shell) 100% POLYESTER(lining...,women's clothing,https://fakestoreapi.com/img/81XH0e8fefL._AC_U...,2.9,340,2024-02-12,0.92825,27.801088
3,16,Lock and Love Women's Removable Hooded Faux Le...,29.95,100% POLYURETHANE(shell) 100% POLYESTER(lining...,women's clothing,https://fakestoreapi.com/img/81XH0e8fefL._AC_U...,2.9,340,2024-02-13,0.92653,27.749573
4,17,Rain Jacket Women Windbreaker Striped Climbing...,39.99,Lightweight perfet for trip or casual wear---L...,women's clothing,https://fakestoreapi.com/img/71HblAHs5xL._AC_U...,3.8,679,2024-02-12,0.92825,37.120718


In [6]:
# I'm using the DuckDB library to mimic a relational database here.
# Check the "DDL_products_big_query.sql" file for the actual table creation script for BigQuery.

# Create statement for the final table
ddl_query = """
        -- Drop the table if it exists
        DROP TABLE IF EXISTS top_products;

        -- Create the table with appropriate data types
        CREATE TABLE top_products (
            id INTEGER,
            title STRING,
            price_usd FLOAT,
            description STRING,
            category STRING,
            image STRING,
            rating_rate FLOAT,
            rating_count INTEGER,
            exchange_rate_date DATE,
            exchange_rate FLOAT,
            price_eur FLOAT
        );
    """
# I'm assuming that the required Top 5 products should be just five items, otherwise this would be a DENSE_RANK().
# In a real scenario this would be improved, for example, using other fields for cases where there are multiple items with the exact same rank.
top_products_query ="""
        INSERT INTO
            top_products
        SELECT
            id,
            title,
            price_usd,
            description,
            category,
            image,
            rating_rate,
            rating_count,
            exchange_rate_date,
            exchange_rate,
            price_eur
        FROM
            (
                SELECT
                    id,
                    title,
                    price_usd,
                    description,
                    category,
                    image,
                    rating_rate,
                    rating_count,
                    exchange_rate_date,
                    exchange_rate,
                    price_eur,
                    rank() OVER (
                        PARTITION BY exchange_rate_date
                        ORDER BY
                            rating_rate DESC
                    ) AS rank
                FROM
                    all_products
            )
        WHERE
            rank <= 5
    """

# Get the data from the "products_df" DataFrame into DuckDB
duckdb.sql("CREATE OR REPLACE TABLE all_products AS SELECT * FROM products_df")
# Create the "top_products" table
duckdb.sql(ddl_query)
# Insert the top products into the "top_products" table using the "top_products_query" query
duckdb.sql(top_products_query)

In [8]:
duckdb.sql("SELECT * FROM top_products").show()

┌───────┬──────────────────────┬───────────┬───┬──────────────┬────────────────────┬───────────────┬────────────┐
│  id   │        title         │ price_usd │ … │ rating_count │ exchange_rate_date │ exchange_rate │ price_eur  │
│ int32 │       varchar        │   float   │   │    int32     │        date        │     float     │   float    │
├───────┼──────────────────────┼───────────┼───┼──────────────┼────────────────────┼───────────────┼────────────┤
│    18 │ MBJ Women's Solid …  │      9.85 │ … │          130 │ 2024-02-12         │       0.92825 │   9.143263 │
│    19 │ Opna Women's Short…  │      7.95 │ … │          146 │ 2024-02-12         │       0.92825 │  7.3795877 │
│    17 │ Rain Jacket Women …  │     39.99 │ … │          679 │ 2024-02-12         │       0.92825 │  37.120716 │
│    20 │ DANVOUY Womens T S…  │     12.99 │ … │          145 │ 2024-02-12         │       0.92825 │  12.057967 │
│    16 │ Lock and Love Wome…  │     29.95 │ … │          340 │ 2024-02-12         │    